In [ ]:
%matplotlib widget
import flammkuchen as fl
import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from lotr import DATASET_LOCATION, LotrExperiment, dataset_folders
from lotr.data_preprocessing.dlc_tracking import export_dlc_behavior
from lotr.utils import convolve_with_tau, interpolate, nan_phase_jumps, zscore
from matplotlib import pyplot as plt
from scipy.stats import ttest_rel, wilcoxon
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm

COLS = pltltr.COLS

In [ ]:
from lotr import DATASET_LOCATION

fish_with_eyes = [
    f.parent
    for f in DATASET_LOCATION.glob("*/*eyes*/selected.h5")
    if "noeyes" not in f.parent.name
]
fish_with_eyes

In [ ]:
def _process_reg(reg):
    reg = convolve_with_tau(reg, int(TAU_S * exp.fs))
    return zscore(reg)


def draw_train_test(n_pts, wnd):
    indexes = np.arange(n_pts - wnd, dtype=np.float)

    # Draw beginning of training window:
    train_i_start = np.random.randint(0, n_pts - wnd)
    train_i_end = train_i_start + wnd

    # Exclude training window from possible choiches:
    indexes[train_i_start:train_i_end] = np.nan
    # Exclude beginning and end if we are to close to limits:
    if train_i_start < wnd:
        indexes[:train_i_start] = np.nan
    if n_pts - train_i_end < wnd:
        indexes[train_i_end:] = np.nan

    indexes = indexes[~np.isnan(indexes)]  # .astype(np.int)
    test_i_start = int(np.random.choice(indexes))

    return slice(train_i_start, train_i_end), slice(test_i_start, test_i_start + wnd)

In [ ]:
np.random.seed(24324215)
TAU_S = 5
WND = 300
results_df = []
for path in tqdm(fish_with_eyes):
    exp = LotrExperiment(path)
    dlc_df = fl.load(path / "behavior_from_dlc.h5", "/data")

    fictive_head = exp.fictive_heading
    phase = exp.network_phase

    eyes_arr = dlc_df["rt_eye_medfilt"] + dlc_df["lf_eye_medfilt"]
    interp_eye = interpolate(dlc_df["t"], eyes_arr, exp.time_arr)

    data_diff_df = pd.DataFrame(
        dict(
            phase=_process_reg(np.diff(np.unwrap(phase))),
            mov_regr=_process_reg(np.diff(fictive_head)),
            eye_pos_regr=_process_reg(np.diff(interp_eye)),
        )
    )
    plt.figure()
    plt.plot(_process_reg(np.diff(np.unwrap(phase))))
    # plt.plot(zscore(interp_eye))
    # plt.plot(zscore(_process_reg(interp_eye)))
    plt.plot(_process_reg(np.diff(interp_eye)))
    plt.plot(_process_reg(np.diff(fictive_head)))

    """
    for to_fit, tofit_lab in zip([data_diff_df], ["data"]):
        # all_coefs = []

        for _ in range(500):
            res_df = dict()
            for lab, cols in zip(
                ["tail", "eye", "both"],
                [["mov_regr"], ["eye_pos_regr"], ["mov_regr", "eye_pos_regr"]],
            ):
                train, test = draw_train_test(len(phase), WND * exp.fs)

                regr = LinearRegression()
                regr.fit(
                    to_fit[cols].values[train, :], to_fit["phase"].values[train],
                )

                # if len(cols) == 2:
                prediction = regr.predict(to_fit[cols].values[test, :])
                # else:
                #    prediction = to_fit[cols[0]].values[test]
                res_df[lab] = np.abs(
                    np.corrcoef(prediction, to_fit["phase"][test])[0, 1]
                )
            # res_df["batch"] = tname
            res_df["data"] = tofit_lab
            res_df["fid"] = path.name

            results_df.append(res_df)
            """

results_df = pd.DataFrame(results_df)

In [ ]:
def _to_float(df):
    df.iloc[:, :].values = df.iloc[:, :].values.astype(np.float)


sel = results_df["data"] == "data"

# for fid in results_df["fid"].unique():
#    selfid = sel & (results_df["fid"] == fid)
# results_df[sel].groupby("fid").quantile((.25, .5, .75))
quantiles_df = (
    results_df.loc[sel, ["both", "tail", "eye", "fid"]]
    .groupby("fid")
    .quantile((0.25, 0.5, 0.75))
)
quantiles_df.index = quantiles_df.index.set_names(["fid", "percentile"])

In [ ]:
f, ax = plt.subplots(figsize=(2.2, 2), gridspec_kw=dict(left=0.2, bottom=0.15, top=0.8))
for i, path in enumerate(fish_with_eyes):
    pltltr.tick_with_bars(
        quantiles_df.xs(path.name, level="fid"),
        cols=[COLS["qualitative"][0]] * 3,
        xdisperse=0.25,
    )
    # plt.plot([np.nan], c=[COLS["qualitative"][0]], label=f"fish {i}")
    # plt.plot(quantiles_df.xs(path.name, level="fid")[0.5].values)
plt.plot(quantiles_df.xs(0.5, level="percentile").values.T, c=".5", lw=0.5)

ax.set(
    ylabel="Fit correlation (cross-val)",
    xticks=[0, 1, 2],
    xticklabels=["eyes+tail", "tail", "eyes"],
    # ylim=(0, 0.8),
)
# ax.legend(loc=2, bbox_to_anchor=(0.7, 1.3), fontsize=6)
pltltr.despine(ax)

pltltr.savefig("eye_tail_correlations")

In [ ]:
wilcoxon(
    quantiles_df.xs(0.5, level="percentile")["both"],
    quantiles_df.xs(0.5, level="percentile")["tail"],
)

In [ ]:
wilcoxon(
    quantiles_df.xs(0.5, level="percentile")["eye"],
    quantiles_df.xs(0.5, level="percentile")["tail"],
)

In [ ]:
ttest_rel(
    quantiles_df.xs(0.5, level="percentile")["both"],
    quantiles_df.xs(0.5, level="percentile")["tail"],
)

In [ ]:
ttest_rel(
    quantiles_df.xs(0.5, level="percentile")["eye"],
    quantiles_df.xs(0.5, level="percentile")["tail"],
)

In [ ]:
ttest_rel(results_df.loc[sel, "both"], results_df.loc[sel, "tail"])

In [ ]:
wilcoxon(
    results_df.loc[sel, "tail"], results_df.loc[sel, "eye"],
)

In [ ]:
results_df

In [ ]:
wilcoxon(
    quantiles_df.xs(0.5, level="percentile")["eye"],
    quantiles_df.xs(0.5, level="percentile")["tail"],
)

In [ ]:
results_df.loc[sel, ["both", "tail", "eye"]]

In [ ]:
path = DATASET_LOCATION / "210601_f0" / "210601_f0_2dvr_eyes"  # fish_with_eyes[1]
exp = LotrExperiment(path)
dlc_df = fl.load(path / "behavior_from_dlc.h5", "/data")

fictive_head = exp.fictive_heading
phase = exp.network_phase

eyes_arr = dlc_df["rt_eye_medfilt"] + dlc_df["lf_eye_medfilt"]
interp_eye = interpolate(dlc_df["t"], eyes_arr, exp.time_arr)

data_diff_df = pd.DataFrame(
    dict(
        phase=_process_reg(np.diff(np.unwrap(phase))),
        mov_regr=_process_reg(np.diff(fictive_head)),
        eye_pos_regr=_process_reg(interp_eye[1:]),
        eye_vel_regr=_process_reg(np.diff(interp_eye)),
    )
)
data_df = pd.DataFrame(
    dict(
        phase=np.unwrap(phase),
        mov_regr=_process_reg(fictive_head),
        eye_pos_regr=_process_reg(interp_eye),
    )
)

In [ ]:
t_slice_s = (1300, 2160)
f, ax = plt.subplots(figsize=(3, 2), gridspec_kw=dict(left=0.2, bottom=0.2, right=0.99))
lw = 1

t_slice = slice(*[t * exp.fs for t in t_slice_s])
t_arr = np.arange(t_slice.stop - t_slice.start) / exp.fs
ax.plot(
    t_arr,
    -zscore(np.unwrap(phase[t_slice])),
    c=COLS["ph_plot"],
    lw=lw,
    label="Network phase",
)
ax.plot(t_arr, zscore(fictive_head[t_slice]), c=COLS["th_plot"], lw=lw, label="Heading")
# twin_ax = ax.twinx()
ax.plot(
    t_arr,
    zscore(interp_eye[t_slice]) / 3 - 3,
    lw=lw,
    c="darkgreen",
    label="Eyes (mean)",
)
ax.set(xlabel="Time (s)")
pltltr.despine(ax, ["left", "top", "right"])
ax.legend(loc=2, bbox_to_anchor=(0.0, 1.1), fontsize=6)
pltltr.savefig("eye_tail_traces")

In [ ]:
exp = LotrExperiment(path)
dlc_df = fl.load(path / "behavior_from_dlc.h5", "/data")

fictive_head = exp.fictive_heading
phase = exp.network_phase

eyes_arr = dlc_df["rt_eye_medfilt"] + dlc_df["lf_eye_medfilt"]
interp_eye = interpolate(dlc_df["t"], eyes_arr, exp.time_arr)

from lotr.utils import pearson_regressors

eye_r_scores = pearson_regressors(exp.traces, interp_eye[:, np.newaxis])[0, :]

In [ ]:
plt.figure(figsize=(6, 5))
plt.plot(exp.time_arr, exp.traces[:, exp.hdn_indexes[::3]])
plt.plot(
    exp.time_arr, zscore(interp_eye) / 3, lw=2, c="k", label="Eyes (mean)",
)
plt.show()

In [ ]:
from lotr.pca import pca_and_phase

pca_scores, angles, _, circle_params = pca_and_phase(
    exp.traces[exp.pca_t_slice, exp.hdn_indexes].T, exp.traces[exp.pca_t_slice, :].T
)
s = 10
plt.figure(figsize=(3, 3))
plt.scatter(pca_scores[:, 0], pca_scores[:, 1], s=s, c=".9")
sel = np.abs(eye_r_scores) > np.percentile(np.abs(eye_r_scores), 90)
plt.scatter(pca_scores[sel, 0], pca_scores[sel, 1], s=s)
plt.scatter(
    pca_scores[exp.hdn_indexes, 0],
    pca_scores[exp.hdn_indexes, 1],
    fc="none",
    ec="r",
    s=s,
    lw=0.5,
)


plt.axis("equal")

In [ ]:
plt.figure(figsize=(3, 3))
beh_df = exp.motor_regressors
plt.scatter(beh_df["left_1"].values - beh_df["right_1"].values, eye_r_scores)
plt.scatter(
    beh_df["left_1"].values[exp.hdn_indexes]
    - beh_df["right_1"].values[exp.hdn_indexes],
    eye_r_scores[exp.hdn_indexes],
)

In [ ]:
TAU_S = 3
WND = 300
results_df = []
for path in tqdm(fish_with_eyes):
    exp = LotrExperiment(path)
    dlc_df = fl.load(path / "behavior_from_dlc.h5", "/data")

    fictive_head = exp.fictive_heading
    phase = exp.network_phase

    eyes_arr = dlc_df["rt_eye_medfilt"] + dlc_df["lf_eye_medfilt"]
    interp_eye = interpolate(dlc_df["t"], eyes_arr, exp.time_arr)

    data_diff_df = pd.DataFrame(
        dict(
            phase=_process_reg(np.diff(np.unwrap(phase))),
            mov_regr=_process_reg(np.diff(fictive_head)),
            eye_pos_regr=_process_reg(interp_eye[1:]),
            eye_vel_regr=_process_reg(np.diff(interp_eye)),
        )
    )
    data_df = pd.DataFrame(
        dict(
            phase=np.unwrap(phase),
            mov_regr=_process_reg(fictive_head),
            eye_pos_regr=_process_reg(interp_eye),
        )
    )

    for to_fit, tofit_lab in zip([data_df], ["data"]):
        # all_coefs = []
        for _ in range(100):
            train, test = draw_train_test(len(phase), WND * exp.fs)

            regr = LinearRegression()
            regr.fit(
                to_fit[["mov_regr", "eye_pos_regr"]].values[train, :],
                to_fit["phase"].values[train],
            )

            prediction = regr.predict(
                to_fit[["mov_regr", "eye_pos_regr"]].values[test, :]
            )

            train_corrs = np.stack(
                [
                    to_fit["phase"][train],
                    pred_fit,
                    to_fit["mov_regr"][train] * regr.coef_[0],
                    to_fit["eye_pos_regr"][train] * regr.coef_[1],
                ]
            )

            test_corrs = np.stack(
                [
                    to_fit["phase"][test],
                    prediction,
                    to_fit["mov_regr"][test] * regr.coef_[0],
                    to_fit["eye_pos_regr"][test] * regr.coef_[1],
                ]
            )
            for t, tname in zip([train_corrs, test_corrs], ["train", "test"]):
                res_df = {
                    k: val
                    for k, val in zip(
                        ["both", "tail", "eye"], np.corrcoef(test_corrs)[0, 1:]
                    )
                }
                res_df["batch"] = tname
                res_df["data"] = tofit_lab
                res_df["fid"] = path.name

                results_df.append(res_df)

results_df = pd.DataFrame(results_df)

In [ ]:
plt.figure()
b = np.arange(-1, 1, 0.02)
plt.hist(eye_r_scores, b, alpha=0.4)
plt.hist(eye_r_scores[exp.hdn_indexes], b, alpha=0.4)
plt.show()

In [ ]:
plt.figure()
plt.plot(exp.time_arr, exp.traces[:, np.argmax(eye_r_scores)])
plt.plot(
    exp.time_arr,
    exp.traces[:, exp.hdn_indexes][:, np.argmax(eye_r_scores[exp.hdn_indexes])],
)
plt.plot(
    exp.time_arr, zscore(interp_eye) / 3, lw=2, c="k", label="Eyes (mean)",
)
plt.plot(
    exp.behavior_log["t"],
    exp.behavior_log["tail_sum"] - 3,
    lw=2,
    c=".4",
    label="Eyes (mean)",
)

In [ ]:
plt.figure()
plt.plot(
    exp.time_arr, exp.network_phase,
)
plt.plot(
    exp.time_arr, zscore(interp_eye) / 3, lw=2, c="k", label="Eyes (mean)",
)
plt.plot(
    exp.behavior_log["t"],
    exp.behavior_log["tail_sum"] - 3,
    lw=2,
    c=".4",
    label="Eyes (mean)",
)